## Install python module for Google Earth Engine

In [1]:
!pip install earthengine-api

## Authenticate the API (follow the instructions and paste the code) (Please replace this line with your code

In [2]:
#Bash cell 
#code = <Please paste the code you get here 
#!earthengine authenticate --authorization-code $code
#!earthengine authenticate --authorization-code 4/bgFz1xBb1MWiAA3THY4ci_VO6kdy5PGBoy27jT3U2RpKfX7tuWxO6_A

In [3]:
import pandas as pd
df = pd.read_csv("point_survey_v1.csv")
df.head()

,district,segment_id,point_id,point_code,su_id,su_date,su_date_year,su_date_month,su_date_day,start_time,...,harvest,harvest_qualifier,d_harv,irrig,lc_remark,photo,remarks,entry_loc,entry_loc_qualifier,end_time
0,42,621,1,42-621-1,D2,10/11/2016,2016,11,10,15:40,...,2.0,NaN,15/02/2017,2.0,NaN,5402_37.jpg,NaN,1,NaN,15:43
1,42,621,2,42-621-2,D2,10/11/2016,2016,11,10,15:50,...,NaN,NaN,null/null/null,NaN,River (flooded area),5442_37.jpg,River bank,1,NaN,15:52
2,42,621,3,42-621-3,D1,10/11/2016,2016,11,10,15:35,...,1.0,NaN,25/10/2016,1.0,NaN,5159_37.jpg,NaN,1,NaN,15:37
3,42,621,4,42-621-4,D1,10/11/2016,2016,11,10,15:42,...,NaN,NaN,null/null/null,NaN,River,5199_37.jpg,NaN,1,NaN,15:44
4,42,645,1,42-645-1,D2,10/11/2016,2016,11,10,14:15,...,NaN,NaN,null/null/null,NaN,NaN,5322_37.jpg,NaN,1,NaN,14:17


## Iterate over 4 rows at a time and build the rectanges to download images for 
Reference: https://stackoverflow.com/questions/39219705/how-to-download-images-using-google-earth-engines-python-ap
Reference: https://developers.google.com/earth-engine/exporting


In [ ]:
import ee
ee.Initialize()
landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515').select(['B4','B3','B2'])

lengthdf = len(df) - 1
#print("length: {}".format(lengthdf))

i = 0
while i <= lengthdf: 
  description = df.loc[i,'segment_id']    
  idx = 0
  c = []

  while (df.loc[i,'segment_id'] == description) and (i<=lengthdf) and (df.loc[i,'segment_id'] != 'null'):
#    print(i, idx )
    c.append([df.loc[i,'coord_obs_x'], df.loc[i,'coord_obs_y'] ])
#    print(c)
    i += 1
    idx += 1

  
  #HACK: NEED TO DO BETTER: It seems there is a bowtie effect. Swap element 3 and 4 in a quadirilateral
  if len(c) == 4:
    a, b = c[2], c[3]
    c[2], c[3] = b, a

  geometry = c 
  print("Geom: {}".format(geometry))
    
  task_config = {
    'image': landsat,
    'description': description,
    'bucket': 'satimages',
    'fileNamePrefix': description,
    'scale': 30,  
    'region': geometry
  }

  task = ee.batch.Export.image.toCloudStorage(**task_config)
  task.start()
  task.status()

Geom: [[84.28586591, 27.42846051], [84.28724124, 27.42850423], [84.28731738, 27.42719899], [84.28578211, 27.42716335]]
Geom: [[84.40704161, 27.43081959], [84.40851456, 27.43078346], [84.40851276, 27.42944815], [84.40694692, 27.42943646]]
Geom: [[84.43769446, 27.43252932], [84.43768928, 27.43267845], [84.43770428, 27.43235336], [84.43776981, 27.43237146]]
Geom: [[84.31585037, 27.43806162], [84.31737319, 27.43811606], [84.31741151, 27.4367469], [84.31589526, 27.43682279]]
Geom: [[84.33603317, 27.43843619], [84.33761965, 27.43853205], [84.33767689, 27.43715581], [84.3360402, 27.43709978]]
Geom: [[84.3410999, 27.438616999999997], [84.30068409999998, 27.477949900000002], [84.34113539, 27.43730537]]
Geom: [[84.29545423, 27.44221338], [84.29708946, 27.44225885], [84.29718276, 27.44082738], [84.29554467, 27.4408718]]
Geom: [[84.35622121, 27.44333103], [84.35766783, 27.44341668], [84.35774926, 27.44203561], [84.3562009, 27.44202807]]
Geom: [[84.30560202, 27.44691468], [84.30652957, 27.44676277]